In [ ]:
""" REDOING entire pipeline. Previuos had too much interaction entretnched in pipeline.
very hard to figure out analyses etc.

Here, outline:
1. Dataset --> strokefeats (one per dataset)
2. Combine strokefeats (arbitrary combo, could incllude parses, beh, beh splines) --> 
get cluster/embeddings data
3. Plots

[OBSOLETE - ALL MOVED TO notebooks_dataset --> analy_strokeclustering_220406

"""

In [1]:
# %cd ..

# from tools.utils import * 
# from tools.plots import *
# from tools.analy import *
# from tools.calc import *
# from tools.analyplot import *
# from tools.preprocess import *
# from tools.dayanalysis import *
# from analysis.strok import *
# from analysis.line2 import *
# from analysis.probedatTaskmodel import *
# from pythonlib.drawmodel.analysis import *
# from pythonlib.tools.stroketools import *

/data1/code/python/drawmonkey


#### Intial extraction of stroke feats (also in strok.py)

In [9]:
from pythonlib.dataset.dataset import Dataset
# expt = "lines5"
# strokes_ver_list = ["strokes_beh_splines", "strokes_parse"]
# animallist = ["Pancho", "Red"]
expt = "gridlinecircle"
# strokes_ver_list = ["strokes_beh_splines", "strokes_parse"]
strokes_ver_list = ["strokes_beh_splines"]
animallist = ["Pancho"]
list_rule = ["baseline", "lolli"]

for a in animallist:
    for rule in list_rule:
        for sver in strokes_ver_list:
            D = Dataset([])
            D.load_dataset_helper(a, expt, ver="single", rule=rule)

            # Extract single strokes feat 
            D.sf_extract_and_save(strokes_ver = sver);



Searching using this string:
/data2/analyses/database//*Pancho*gridlinecircle*baseline*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
0
Searching using this string:
/data2/analyses/database/BEH/*Pancho*gridlinecircle*baseline*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
1
---
/data2/analyses/database/BEH/Pancho-gridlinecircle-baseline-210824_002447
----------------
Currently loading: /data2/analyses/database/BEH/Pancho-gridlinecircle-baseline-210824_002447
Loaded metadat:
{'sketchpad_edges': array([[-311.84, -224.8 ],
       [ 311.84,  429.6 ]]), 'metadat_probedat': {'sdate': 210820, 'edate': 210821, 'strokmodel_kind': None, 'strokmodel_tstamp': None, 'datecategories': {'210820': 1, '210821': 1}, 'dates_for_summary': [], 'matchedstrokes': None, 'exptnames': ['gridlinecircle1', 'gridlinecircle2'], 'T1': [], 'G1': [], 'G2': [], 'G3': [], 'G4': [], 'description': '', 'finalized': False, 'good_expt': True, 'expt': 'gridlinecircle', 'task_train_test':

In [10]:
# first get all combinations
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

animallist_list = list(powerset(animallist))[1:]
strokesverlist_list = list(powerset(strokes_ver_list))[1:]


In [13]:
print(animallist_list)
print(strokes_ver_list)

[('Pancho',)]
('strokes_beh_splines',)


##### Embedding in sim matrix
##### Cluster sim matrix

In [17]:
sdir

'/data2/analyses/database/combined_strokfeats/Pancho-gridlinecircle-strokes_beh_splines-210923_210307/embeddings/similarity-stretch_to_1-euclidian_diffs-N5-210923_210307/clustering'

In [16]:
DEBUG = True
import os
list_rule = ["baseline", "lolli"]

# Run analysis for all single and combined datasets
for animallist in animallist_list:
    for strokes_ver_list in strokesverlist_list:
        if "Pancho" in animallist and "Red" in animallist:
            # then skip some things.
#             if len(strokes_ver_list)==1 and "strokes_parse" in strokes_ver_list:
#                 continue
            if len(strokes_ver_list)==1:
                continue

        ### now load multiple datasets, and concat SF, preprocess and save
        D = Dataset([])
        D.load_dataset_helper(animallist, expt, "mult", rule=list_rule)
        D.sf_load_preextracted(strokes_ver_list=strokes_ver_list)

        if DEBUG:
            D.SF = D.SF[:100]

        # preprocess SF
        D.sf_preprocess_stroks()
        D.sf_save_combined_sf()

        ### FINAL LOAD BEFORE GET EMBEDDINGS
        D = Dataset([])
        D.load_dataset_helper(animallist, expt, "mult", rule=list_rule)
        D.sf_load_combined_sf(animallist, [expt], strokes_ver_list)

        # Load SFs, and perform embeddings and clustering
        if DEBUG:
            similarity_matrix, idxs_stroklist_basis, params = D.sf_embedding_bysimilarity(Nbasis=5)
        else:
            similarity_matrix, idxs_stroklist_basis, params = D.sf_embedding_bysimilarity()

        ### MODEL the sim matrix
        from pythonlib.tools.clustertools import clusterSimMatrix

        if DEBUG:
            DAT = clusterSimMatrix(similarity_matrix, gmm_n_mixtures=[4], perplist = [35])
        else:
            DAT = clusterSimMatrix(similarity_matrix, gmm_n_mixtures, perplist)

        # Save clustering results
        import pickle

        s = params["path_embeddings_similarity"]
        sdir = f"{s}/clustering"
        os.makedirs(sdir, exist_ok=True)

        path = f"{sdir}/DAT.pkl"
        with open(path, "wb") as f:
            pickle.dump(DAT, f)
        print(path)

Searching using this string:
/data2/analyses/database//*Pancho*gridlinecircle*baseline*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
0
Searching using this string:
/data2/analyses/database/BEH/*Pancho*gridlinecircle*baseline*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
1
---
/data2/analyses/database/BEH/Pancho-gridlinecircle-baseline-210824_002447
Searching using this string:
/data2/analyses/database//*Pancho*gridlinecircle*lolli*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
0
Searching using this string:
/data2/analyses/database/BEH/*Pancho*gridlinecircle*lolli*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
1
---
/data2/analyses/database/BEH/Pancho-gridlinecircle-lolli-210903_094051
----------------
Currently loading: /data2/analyses/database/BEH/Pancho-gridlinecircle-baseline-210824_002447
Loaded metadat:
{'sketchpad_edges': array([[-311.84, -224.8 ],
       [ 311.84,  429.6 ]]), 'metadat_probedat': {'sda

Deleted unused columns from self.Dat
applying monkey train test names
resetting index
Removed online aborts
Updated columns: insummarydates, using Metadats
Searching using this string:
/data2/analyses/database/combined_strokfeats/*Pancho-gridlinecircle-strokes_beh_splines**/*SF*pkl
-- Splitting off dir from fname
Found this many paths:
4
---
/data2/analyses/database/combined_strokfeats/Pancho-gridlinecircle-strokes_beh_splines-210923_210101
---
/data2/analyses/database/combined_strokfeats/Pancho-gridlinecircle-strokes_beh_splines-210923_210234
---
/data2/analyses/database/combined_strokfeats/Pancho-gridlinecircle-strokes_beh_splines-210923_210254
---
/data2/analyses/database/combined_strokfeats/Pancho-gridlinecircle-strokes_beh_splines-210923_210307
Taking most recent path
==== Loading:
** Loaded into self.SF:
/data2/analyses/database/combined_strokfeats/Pancho-gridlinecircle-strokes_beh_splines-210923_210307/SF.pkl
** Loaded into self.SFparams:
/data2/analyses/database/combined_strokf

### PLOTTING

In [213]:
# Load data (dataset + SF)
animallist = ["Pancho"]
strokes_ver_list = ["strokes_beh_splines", "strokes_parse"]

D = Dataset([])
D.load_dataset_helper(animallist, expt, "mult")
D.sf_load_combined_sf(animallist, [expt], strokes_ver_list)


# Load clustering data
D.SFparams["path_sf_combined"]


pathlist = findPath(D.SFparams["path_sf_combined"], 
         [["embeddings"], ["similarity", "stretch_to_1", "euclidian_diffs"]], "", "", True)

if len(pathlist)>1:
    assert False
    
pathdat = f"{pathlist[0]}/dat.pkl"
with open(pathdat, "rb") as f:
    dat = pickle.load(f)
    
pathdat = f"{pathlist[0]}/params.pkl"
with open(pathdat, "rb") as f:
    params = pickle.load(f)


params["path_embeddings_similarity"]

Did not load data!!!
Searching using this string:
/data2/analyses/database//*Pancho*lines5*/*dat*.pkl
-- Splitting off dir from fname
Found this many paths:
1
---
/data2/analyses/database/Pancho-lines5-formodeling-210329_014835
----------------
Currently loading: /data2/analyses/database/Pancho-lines5-formodeling-210329_014835
Loaded dataset, size:
6949
Loaded metadat:
{'sketchpad_edges': array([[-327.2, -327.2],
       [ 327.2,  429.6]])}
----
Resetting index
=== CLEANING UP self.Dat ===== 
ORIGINAL: online abort values
Series([], Name: online_abort, dtype: int64)
kept 6949 out of 6949
removed all cases with online abort not None
Deleted unused columns from self.Dat
Searching using this string:
/data2/analyses/database/combined_strokfeats/*Pancho-lines5-strokes_beh_splines*strokes_parse**/*SF*pkl
-- Splitting off dir from fname
Found this many paths:
1
---
/data2/analyses/database/combined_strokfeats/Pancho-lines5-strokes_beh_splines_strokes_parse-210412_175537
==== Loading:
** Loaded